In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
titanic_df = pd.read_csv('datasets/titanic_processed.csv')

titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,1,1,45.0,1,0,83.4750,0,0,1
1,0,3,1,42.0,0,1,8.4042,0,0,1
2,1,3,1,1.0,1,2,20.5750,0,0,1
3,0,3,1,23.5,0,0,7.2292,1,0,0
4,1,3,1,27.0,0,0,8.6625,0,0,1


In [5]:
FEATURES = list(titanic_df.columns[1:])
FEATURES

['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

In [6]:
result_dict={}

In [7]:
def summarize_classification(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred, normalize=True)
    num_acc = accuracy_score(y_test, y_pred, normalize=False) 
    prec = precision_score(y_test, y_pred) 
    rec = recall_score(y_test, y_pred)
    
    return {
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'accuracy_count': num_acc
    }

In [9]:
def build_model(classifier_fn,
               label,
               features,
               dataset,
               test_frac=0.2):
    X = dataset[features]
    Y = dataset[label]
    
    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=test_frac)
    
    model = classifier_fn(x_train, y_train)
    
    y_pred = model.predict(x_test)
    y_pred_train = model.predict(x_train)
    
    test_summary = summarize_classification(y_test, y_pred)
    train_summary = summarize_classification(y_train, y_pred_train)
    
    pred_results = pd.DataFrame({'y_test': y_test,
                                 'y_pred': y_pred})
    
    model_crosstab = pd.crosstab(pred_results.y_pred, pred_results.y_test)
    
    return {'training': train_summary,
           'test': test_summary,
           'confusion_matrix': model_crosstab}

In [13]:
def compare_results():
    for key in result_dict:
        print('Classification: ', key)
        
        print()
        print('Training data')
        for score in result_dict[key]['training']:
            print(score, result_dict[key]['training'][score])
            
        print()
        print('Test data')
        for score in result_dict[key]['test']:
            print(score, result_dict[key]['test'][score])
            
        print()

In [11]:
def logistic_fn(x_train, y_train):
    
    model = LogisticRegression(solver='liblinear')
    model.fit(x_train, y_train)
    
    return model

In [14]:
result_dict['logistic'] = build_model(logistic_fn,
                                     'Survived',
                                     FEATURES,
                                     titanic_df)

compare_results()

Classification:  logistic

Training data
accuracy 0.7961335676625659
precision 0.7789473684210526
recall 0.6666666666666666
accuracy_count 453

Test data
accuracy 0.7902097902097902
precision 0.7903225806451613
recall 0.7424242424242424
accuracy_count 113

